# 分子数据转换为图结构

## 1. 数据提取

In [1]:
# 读取 QM9_demo 数据集
from utils.data import QM9DataExtractor

# 使用定义的QM9DataExtractor类用于加载提取数据
extractor = QM9DataExtractor('../data/demo/qm9_demo.csv')

**测试数据提取功能**

In [2]:
# 使用 SMILES 字符串 "CC(C)=O"（丙酮）测试数据提取功能
info = extractor.get_molecule_info('CC(C)=O')
print("数据结构:")
print("坐标数据keys:", info['coordinates_data'].keys())
print("属性数据keys:", info['properties_data'].keys())
info

数据结构:
坐标数据keys: dict_keys(['smiles_gdb', 'atoms', 'coordinates', 'charges', 'num_atoms'])
属性数据keys: dict_keys(['smiles_gdb', 'properties', 'property_vector'])


{'coordinates_data': {'smiles_gdb': 'CC(C)=O',
  'atoms': ['C', 'C', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H'],
  'coordinates': array([[-3.10189430e-03,  1.48050328e+00, -1.72598667e-01],
         [-4.52720177e-02, -2.88705905e-02,  1.39492550e-03],
         [ 1.29705800e+00, -7.21944146e-01,  1.68890419e-01],
         [-1.08643172e+00, -6.42733433e-01,  6.30393310e-03],
         [ 5.94035295e-01,  1.74668579e+00, -1.05225043e+00],
         [-1.01540407e+00,  1.87034023e+00, -2.83822650e-01],
         [ 4.80995996e-01,  1.95070605e+00,  6.90993759e-01],
         [ 1.82512144e+00, -3.24707309e-01,  1.04328658e+00],
         [ 1.14892507e+00, -1.79597689e+00,  2.85543930e-01],
         [ 1.93682635e+00, -5.29496921e-01, -6.99954028e-01]]),
  'charges': array([-0.468686,  0.391521, -0.468685, -0.28561 ,  0.136467,  0.142771,
          0.136492,  0.136469,  0.142771,  0.136491]),
  'num_atoms': 10},
 'properties_data': {'smiles_gdb': 'CC(C)=O',
  'properties': {'A': 10.12193,
   'B': 8.4901

## 2. 图结构转换

**节点、边特征定义**
- 节点：
1. 原子类型（one-hot编码, H, C, N, O, F）
2. 原子坐标（x, y, z）
3. 原子电荷
4. 原子度数（RDkit）
5. 杂化类型（RDkit）

- 边：
1. 几何距离（键长）
2. 化学键类型（RDkit）
3. 共轭性（RDkit）
4. 立体化学信息（RDkit）

In [4]:
# 导入图转换器
from utils.data import MoleculeToGraph

# 创建图转换器实例
graph_converter = MoleculeToGraph()

# 使用丙酮分子测试
molecule_data = extractor.get_molecule_info('CC(C)=O')
smiles = 'CC(C)=O'

# 转换为图结构
graph_data = graph_converter.convert_to_graph(molecule_data, smiles)

print("图结构信息:")
print(f"节点数量: {graph_data['num_nodes']}")
print(f"边数量: {graph_data['num_edges']}")
print(f"节点特征维度: {graph_data['x'].shape}")
print(f"边索引形状: {graph_data['edge_index'].shape}")
print(f"边特征形状: {graph_data['edge_attr'].shape}")

print("\n节点特征详情:")
print("特征组成: 原子类型(5) + 坐标(3) + 电荷(1) + 度数(1) + 杂化类型(7) = 17维")
print("节点特征矩阵:")
print(graph_data['x'])

图结构信息:
节点数量: 10
边数量: 18
节点特征维度: torch.Size([10, 17])
边索引形状: torch.Size([2, 18])
边特征形状: torch.Size([18, 7])

节点特征详情:
特征组成: 原子类型(5) + 坐标(3) + 电荷(1) + 度数(1) + 杂化类型(7) = 17维
节点特征矩阵:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -3.1019e-03,  1.4805e+00, -1.7260e-01, -4.6869e-01,  4.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -4.5272e-02, -2.8871e-02,  1.3949e-03,  3.9152e-01,  3.0000e+00,
          0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.2971e+00, -7.2194e-01,  1.6889e-01, -4.6869e-01,  4.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

In [5]:
print("边特征详情:")
print("特征组成: 几何距离(1) + 化学键类型(4) + 共轭性(1) + 立体化学(1) = 7维")
print("边索引 (源节点 -> 目标节点):")
print(graph_data['edge_index'].T)
print("\n边特征矩阵:")
print(graph_data['edge_attr'])

print("\n节点特征分解 (以第一个节点为例):")
node_0_features = graph_data['x'][0]
print(f"原子类型 (one-hot): {node_0_features[:5].numpy()}")
print(f"坐标 (x,y,z): {node_0_features[5:8].numpy()}")
print(f"电荷: {node_0_features[8].numpy()}")
print(f"度数: {node_0_features[9].numpy()}")
print(f"杂化类型 (one-hot): {node_0_features[10:17].numpy()}")

边特征详情:
特征组成: 几何距离(1) + 化学键类型(4) + 共轭性(1) + 立体化学(1) = 7维
边索引 (源节点 -> 目标节点):
tensor([[0, 1],
        [1, 0],
        [1, 2],
        [2, 1],
        [1, 3],
        [3, 1],
        [0, 4],
        [4, 0],
        [0, 5],
        [5, 0],
        [0, 6],
        [6, 0],
        [2, 7],
        [7, 2],
        [2, 8],
        [8, 2],
        [2, 9],
        [9, 2]])

边特征矩阵:
tensor([[1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2087, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2087, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0960, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0960, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0905, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0905, 1.0000, 0.0000, 0

## 3. 特征提取总结

**节点特征 (17维):**
1. 原子类型: one-hot编码 (H, C, N, O, F) - 5维
2. 原子坐标: (x, y, z) - 3维  
3. 原子电荷: 来自QM9数据集 - 1维
4. 原子度数: 从RDKit获取 - 1维
5. 杂化类型: one-hot编码 - 7维

**边特征 (7维):**
1. 几何距离: 根据原子坐标计算 - 1维
2. 化学键类型: one-hot编码 (单键、双键、三键、芳香键) - 4维
3. 共轭性: 布尔值 - 1维  
4. 立体化学信息: 是否在环中 - 1维

**实现特点:**
- 结合QM9原始数据(坐标、电荷)和RDKit化学信息
- 支持仅使用坐标数据的回退模式
- 输出PyTorch Geometric兼容的数据格式